# Convex quantile/expectile regression

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Objective: Estimates a convex quantile/expectile regression model
   + Data : Timo Kuosmanen (timo.kuosmanen@aalto.fi)
   + Date : April 16, 2020

References:

[1] Kuosmanen, T., Zhou, X. and Dai, S. (2020) How much climate policy has cost for OECD countries?, World Development, 125, p. 104681.

[2] Dai, S., Zhou, X. and Kuosmanen, T. (2920) Forward-looking assessment of the GHG abatement cost: Application to China, Energy Economics, Forthcoming.

In [2]:
import CQER
import pandas as pd
import numpy as np

In [3]:
# import data
df = pd.read_excel ('data.xlsx')

In [4]:
# output (mean scaling before log transformation)
y  = df['TOTEX']
y  = y.to_numpy()
y  = y.tolist()

# inputs
x1  = df['Energy']
x1  = np.asmatrix(x1).T
x2  = df['Length']
x2  = np.asmatrix(x2).T
x3  = df['Customers']
x3  = np.asmatrix(x3).T
x   = np.concatenate((x1, x2, x3), axis=1)
x   = x.tolist()

In [5]:
# calculate the model
from pyomo.environ import *
from pyomo.opt import SolverFactory
crt   = "mult"
func  = "cost"
pps   = "crs"
tau   = 0.7
tile  = "expectile"

model = CQER.cqer(y, x, tau, crt, func, pps, tile)

solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

Note: when estimating the multiplicative model, we can choose the solver such as `knitro` or `minos`. When calculating the additive model, we can choose the solver such as `cplex` or `mosek`.

In [6]:
# display estiamtes
model.em.display()
model.ep.display() 
model.b.display() 

em : error term eminus
    Size=89, Index=i
    Key : Lower : Value                 : Upper : Fixed : Stale : Domain
      0 :  None : -0.025143351885313987 :  None : False : False :  Reals
      1 :  None :  -0.01822415381243636 :  None : False : False :  Reals
      2 :  None :  -0.14153834406489488 :  None : False : False :  Reals
      3 :  None : 0.0031634432895112437 :  None : False : False :  Reals
      4 :  None :  -0.12200994877229607 :  None : False : False :  Reals
      5 :  None :   0.08338539744848716 :  None : False : False :  Reals
      6 :  None :  0.007911872125481735 :  None : False : False :  Reals
      7 :  None :  -0.08345397874794341 :  None : False : False :  Reals
      8 :  None :  -0.26158858651739725 :  None : False : False :  Reals
      9 :  None :  0.028129582859018273 :  None : False : False :  Reals
     10 :  None :  -0.01612809655434067 :  None : False : False :  Reals
     11 :  None :   0.05445487638662762 :  None : False : False :  Reals
     12

In [7]:
# retrive the residuals (eminos)
val = list(model.em[:].value)
eminos = np.asarray(val)
eminos

array([-0.02514335, -0.01822415, -0.14153834,  0.00316344, -0.12200995,
        0.0833854 ,  0.00791187, -0.08345398, -0.26158859,  0.02812958,
       -0.0161281 ,  0.05445488, -0.17353011, -0.18173447,  0.01759172,
        0.10120918, -0.13646847, -0.11849268,  0.00969052,  0.069105  ,
        0.08887631,  0.18235999, -0.1309102 ,  0.06582619, -0.08506866,
       -0.04545292, -0.17058745,  0.13361699, -0.07369125, -0.0194023 ,
        0.00108988,  0.30179505,  0.03156923, -0.15116863,  0.00281435,
       -0.11197111,  0.15115376,  0.10686711,  0.08053498,  0.02593342,
        0.03377751, -0.03159355, -0.04551038,  0.04328749,  0.0624651 ,
        0.09158347, -0.01912621, -0.05882231,  0.09611666,  0.09281379,
       -0.1372429 ,  0.0159075 , -0.14028459, -0.13992518, -0.03001163,
        0.15257954,  0.16080909,  0.06203085,  0.13323596, -0.01416289,
        0.14992125,  0.05626597,  0.02503233, -0.01086764, -0.25615741,
        0.07665536, -0.05686717,  0.03419892, -0.05752511,  0.16

In [8]:
# retrive the residuals (eplus)
val = list(model.ep[:].value)
eplus = np.asarray(val)
eplus

array([ 0.01077572,  0.00781035,  0.06065929, -0.00135576,  0.05228998,
       -0.0357366 , -0.0033908 ,  0.03576599,  0.11210939, -0.01205554,
        0.00691204, -0.0233378 ,  0.07437005,  0.0778862 , -0.00753931,
       -0.04337536,  0.05848649,  0.05078258, -0.00415308, -0.02961643,
       -0.03808985, -0.07815428,  0.05610437, -0.02821123,  0.036458  ,
        0.01947982,  0.07310891, -0.05726442,  0.03158196,  0.00831527,
       -0.00046709, -0.12934073, -0.01352967,  0.06478655, -0.00120615,
        0.04798762, -0.06478018, -0.04580019, -0.03451499, -0.01111432,
       -0.01447608,  0.01354009,  0.01950445, -0.01855178, -0.02677076,
       -0.03925006,  0.00819695,  0.02520956, -0.04119285, -0.03977734,
        0.05881839, -0.0068175 ,  0.06012197,  0.05996794,  0.01286213,
       -0.06539123, -0.06891818, -0.02658465, -0.05710113,  0.00606981,
       -0.06425196, -0.02411399, -0.01072814,  0.00465756,  0.10978175,
       -0.0328523 ,  0.02437165, -0.01465668,  0.02465362, -0.07

In [11]:
# retrive the beta
ind = list(model.b)
val = list(model.b[:,:].value)
beta= np.asarray([ i + tuple([j]) for i, j in zip(ind, val)]) 

import pandas as pd
beta = pd.DataFrame(beta, columns = ['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2', 'b3']

beta

,b1,b2,b3
Name,,,
0.0,8.334386,0.998662,1.078898e-02
1.0,0.000005,0.942649,1.150840e-01
2.0,5.016329,0.963974,5.778983e-02
3.0,6.923478,1.107816,7.295054e-03
4.0,6.887230,1.139709,8.643191e-10
...,...,...,...
84.0,8.334402,0.998662,1.078872e-02
85.0,8.334076,0.998648,1.079591e-02
86.0,8.334329,0.998658,1.079054e-02
